# Unsupervised Alignment of Mouse Neural Representations

## Overview
- このノートブックではNeural RepresentationsのUnsupervised Alignmentの実行方法に付いて説明する。
- 実行ファイルはすべて`scripts`内に格納されている。

In [ ]:
## Group Alignment

### Alignment
#### 処理詳細

#### 実行方法


### Evaluation
#### 実行方法

#### 評価方法


In [ ]:
## 処理詳細
-